# Text-to-SQL Using DuckDB-NSQL

This can be slow when done using CPU only. On a 4-core 16GB codepsace, it takes about 4min 22s to run.


[DuckDB-NSQL GitHub](https://github.com/NumbersStationAI/DuckDB-NSQL)

[DuckDB-NSQL-7B-v0.1-GGUF](https://huggingface.co/motherduckdb/DuckDB-NSQL-7B-v0.1-GGUF)

## 1. Setup

### 1.1 Ensure you are using the correct version of Python

This example uses Python 3.11.8, which should be set automatically if you are using the devcontainer.

In [1]:
!python --version

Python 3.11.8


### 1.2 Set Environment Variables for llama-cpp-python

In [2]:
#!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

### 1.3 Download DuckDB-NSQL GGUF Model

Make sure you comment out the followoing for subsequent runs, otherwise the model will download everytime.

In [3]:
#!huggingface-cli download motherduckdb/DuckDB-NSQL-7B-v0.1-GGUF DuckDB-NSQL-7B-v0.1-q8_0.gguf --local-dir . --local-dir-use-symlinks False

## 2. Model Execution

### 2.1 Import Libraries

In [4]:
import os
import tomllib
import duckdb

from wurlitzer import pipes
from utils import generate_sql
from llama_cpp import Llama

### 2.2 Load Model 

In [5]:
with pipes() as (out, err):
    client = Llama(
        model_path="DuckDB-NSQL-7B-v0.1-q8_0.gguf",
        n_ctx=2048,
    )

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from DuckDB-NSQL-7B-v0.1-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32      

llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32016]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32016]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 2
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - t

### 2.3 Connect to MotherDuck

In [6]:
# Load the TOML file for MotherDuck credentials
with open("config.toml", "rb") as file:
    config = tomllib.load(file)

# Extract the MD_SERVICE_TOKEN from the loaded config
md_service_token = config['mother-duck']['MD_SERVICE_TOKEN']

os.environ['motherduck_token'] = md_service_token

con = duckdb.connect(f'md:?motherduck_token={md_service_token}') 

### 2.4 Prompt SQL Generation

In [7]:
question = "Search for all concepts within the voc_omop54_shared.concept table that contain the word 'Thiamine'"

In [8]:
%%time
sql_query = generate_sql(question, con, client)


llama_print_timings:        load time =   79007.28 ms
llama_print_timings:      sample time =       4.54 ms /    26 runs   (    0.17 ms per token,  5729.40 tokens per second)
llama_print_timings: prompt eval time =  124144.84 ms /   799 tokens (  155.38 ms per token,     6.44 tokens per second)
llama_print_timings:        eval time =    6947.22 ms /    25 runs   (  277.89 ms per token,     3.60 tokens per second)
llama_print_timings:       total time =  131164.15 ms /   824 tokens


!!!Invalid SQL detected!!!
 SELECT * FROM voc_omop54_shared.concept WHERE concept_name LIKE '%Thiamine%';
Exception: Catalog Error: Column with name concept_id already exists!

CPU times: user 4min 22s, sys: 562 ms, total: 4min 22s
Wall time: 2min 11s


In [9]:
print(sql_query)

 SELECT * FROM voc_omop54_shared.concept WHERE concept_name LIKE '%Thiamine%';


### 2.5 Direct Execution on Database

In [10]:
%%time
con.execute(sql_query).fetchdf()

CPU times: user 28.5 ms, sys: 3.99 ms, total: 32.5 ms
Wall time: 188 ms


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45053776,Ascorbic Acid 60 MG / Folic Acid 0.3 MG / Niac...,Drug,NDC,11-digit NDC,None,54569462900,2013-01-01,2016-08-01,D
1,45059563,Thiamine 100 MG Oral Tablet,Drug,NDC,11-digit NDC,None,00002206302,2007-06-01,2011-01-01,D
2,45061948,Thiamine 200 MG/ML Injectable Solution,Drug,NDC,11-digit NDC,None,00381008630,2007-06-01,2011-01-01,D
3,45062388,Thiamine 50 MG Oral Tablet,Drug,NDC,11-digit NDC,None,00527113610,2007-06-01,2011-01-01,D
4,45062464,Thiamine 50 MG Oral Tablet,Drug,NDC,11-digit NDC,None,00536467810,2007-06-01,2011-01-01,D
...,...,...,...,...,...,...,...,...,...,...
11240,45045956,Thiamine 100 MG/ML Injectable Solution,Drug,NDC,11-digit NDC,None,00736008530,2007-06-01,2011-01-01,D
11241,45046589,"Beta Carotene, Ascorbic Acid, Cholecalciferol,...",Drug,NDC,9-digit NDC,None,013925127,2013-08-05,2099-12-31,None
11242,45048725,Thiamine 100 MG Oral Tablet,Drug,NDC,11-digit NDC,None,12634030401,2007-06-01,2011-01-01,D
11243,45049141,Thiamine 100 MG Oral Tablet,Drug,NDC,11-digit NDC,None,17236045601,2007-06-01,2011-01-01,D
